In [1]:
import numpy as np
import tensorflow as tf

In [2]:
x_a = np.array([1, 2, 3], np.float32)
d_a = np.array([-5,3,1], np.float32)

In [3]:
x_b = np.array([1,2,5], np.float32)
d_b = np.array([-7,1,0], np.float32)

In [4]:
class MyModel(tf.keras.Model):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    # Start the gradient walk at the following parameters:
    # In practice, these should be randomly initialized
    self.t = param_t = tf.Variable(-1.0, trainable=True)
    self.s = param_s = tf.Variable(-1.0, trainable=True)
    self.system_tensor = tf.constant([d_a, d_b, x_a, x_b])
    self.last_err = tf.Variable(0.0)
    
  def __call__(self, x, training=False):
    a_pt = self.t * self.system_tensor[0] + self.system_tensor[2]
    b_pt = self.s * self.system_tensor[1] + self.system_tensor[3]
    diff_vec = a_pt - b_pt
    self.last_err = tf.tensordot(diff_vec, diff_vec, axes=1)
    return self.last_err

In [5]:
def fit_dump_model(epoch_count):
    model = MyModel()

    # List the variables tf.modules's built-in variable aggregation.
    print("Variables:", model.variables)

    model.compile(
        run_eagerly=False,
        optimizer=tf.keras.optimizers.Adadelta(learning_rate=1),
        loss=tf.keras.losses.mean_squared_error
        
    )
    model.fit([0], [0], epochs=epoch_count, batch_size=1, verbose=0,
             callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss',
                                                      patience=1,
                                                      mode='min',
                                                      restore_best_weights=True)])
    
    print(model.t)
    print(model.s)


print("For 10,000 epochs")
fit_dump_model(10000)

For 10,000 epochs
Variables: [<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=-1.0>, <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=-1.0>, <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0>]
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.32543072>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.24736899>
